In [9]:
# 중심축 기반 부피 계산
import trimesh
import numpy as np
from scipy.spatial.transform import Rotation
from scipy.optimize import least_squares

# 상수 정의
THICKNESS = 0.0003
MIN_POINTS_PER_SLICE = 10
MAX_CONSECUTIVE_FAILURES = 3
MIN_RADIUS, MAX_RADIUS = 0.02, 0.1
MIN_POINTS_PER_METER = 50.0

# GLB 로드 및 회전
def load_and_align_scene(filepath):
    scene = trimesh.load(filepath)
    A = np.array(scene.metadata.get('hf_alignment', np.eye(4))).reshape(4, 4)
    gravity = (A[:3, :3] @ np.array([[1,0,0],[0,-1,0],[0,0,-1]]) @ np.array([0,1,0]))
    gravity = gravity / np.linalg.norm(gravity)
    
    R = Rotation.align_vectors([[0, 0, 1]], [gravity])[0]
    T = np.eye(4)
    T[:3, :3] = R.as_matrix()
    for geom in scene.geometry.values():
        geom.apply_transform(T)
    
    for geometry in scene.geometry.values():
        if isinstance(geometry, trimesh.PointCloud):
            return geometry.vertices
    return None

# 원 검출 함수들
def refine_circle_ls(inlier_points):
    if len(inlier_points) < 3:
        return None
    center_init = inlier_points.mean(axis=0)
    radius_init = np.mean(np.linalg.norm(inlier_points - center_init, axis=1))
    
    def residuals(params):
        cx, cy, r = params
        return np.linalg.norm(inlier_points - [cx, cy], axis=1) - r
    
    try:
        result = least_squares(residuals, [center_init[0], center_init[1], radius_init],
                              bounds=([-0.5, -0.5, 0.001], [0.5, 0.5, 0.2]))
        cx, cy, r = result.x
        if 0.01 < r < 0.15:
            return {'center': np.array([cx, cy]), 'radius': r}
    except:
        pass
    return None

def fit_circle_ransac(points_2d, n_iter=50, threshold=0.002, min_inliers=10):
    if len(points_2d) < 3:
        return None
    
    best_circle, best_score = None, 0
    for _ in range(n_iter):
        p1, p2, p3 = points_2d[np.random.choice(len(points_2d), 3, replace=False)]
        try:
            A = np.array([[2*(p2[0]-p1[0]), 2*(p2[1]-p1[1])],
                         [2*(p3[0]-p1[0]), 2*(p3[1]-p1[1])]])
            b = np.array([p2[0]**2 - p1[0]**2 + p2[1]**2 - p1[1]**2,
                         p3[0]**2 - p1[0]**2 + p3[1]**2 - p1[1]**2])
            center = np.linalg.solve(A, b)
            radius = np.linalg.norm(p1 - center)
            
            if not (0.01 < radius < 0.15):
                continue
            
            distances = np.abs(np.linalg.norm(points_2d - center, axis=1) - radius)
            inliers = distances < threshold
            n_inliers = np.sum(inliers)
            
            if n_inliers < min_inliers:
                continue
            
            score = n_inliers / len(points_2d)
            if score > best_score:
                best_score = score
                refined = refine_circle_ls(points_2d[inliers])
                if refined:
                    best_circle = {**refined, 'score': score, 'n_inliers': n_inliers}
        except np.linalg.LinAlgError:
            continue
    return best_circle

def detect_circle_at_axis(slice_2d, center_axis, n_iter=100, threshold=0.0003):
    if len(slice_2d) < MIN_POINTS_PER_SLICE:
        return None
    
    distances = np.linalg.norm(slice_2d - center_axis, axis=1)
    best_radius, best_ppm = None, 0.0
    
    for _ in range(n_iter):
        r_candidate = distances[np.random.randint(len(slice_2d))]
        if not (MIN_RADIUS <= r_candidate <= MAX_RADIUS):
            continue
        
        inlier_count = np.sum(np.abs(distances - r_candidate) < threshold)
        circumference = 2 * np.pi * r_candidate
        ppm = inlier_count / circumference if circumference > 0 else 0
        
        if ppm > best_ppm:
            best_ppm, best_radius = ppm, r_candidate
    
    if best_radius and best_ppm >= MIN_POINTS_PER_METER:
        return best_radius
    return None

# 유틸리티 함수들
def get_slice(points, z, thickness=THICKNESS):
    """높이 z에서 슬라이스 추출"""
    mask = (points[:, 2] >= z - thickness) & (points[:, 2] <= z + thickness)
    return points[mask]

def add_volume_data(volume_data, z, radius):
    """volume_data에 단면 정보 추가"""
    volume_data.append({'z': z, 'radius': radius, 'area': np.pi * radius**2})

def search_heights(points, z_range, center_axis, volume_data, max_failures=MAX_CONSECUTIVE_FAILURES):
    """높이 범위를 탐색하며 원 검출"""
    failures = 0
    for z in z_range:
        slice_points = get_slice(points, z)
        if len(slice_points) < MIN_POINTS_PER_SLICE:
            failures += 1
            if failures >= max_failures:
                break
            continue
        
        radius = detect_circle_at_axis(slice_points[:, :2], center_axis)
        if radius is not None:
            failures = 0
            add_volume_data(volume_data, z, radius)
        else:
            failures += 1
            if failures >= max_failures:
                break

# 메인 로직
points = load_and_align_scene('/data/ephemeral/home/project/output/scene.glb')

# 전처리: 원점 근처 필터링 및 정렬
mask = np.linalg.norm(points[:, :2], axis=1) < 0.2
filtered = points[mask]
sorted_idx = np.argsort(filtered[:, 2])
sorted_points, sorted_heights = filtered[sorted_idx], filtered[sorted_idx, 2]

# 1단계: 중심축 계산
z_range = np.linspace(sorted_heights.min(), sorted_heights.max(), 20)
circle_data = []
origin = np.array([0, 0])

for z in z_range:
    start = np.searchsorted(sorted_heights, z - THICKNESS)
    end = np.searchsorted(sorted_heights, z + THICKNESS)
    if end - start < MIN_POINTS_PER_SLICE:
        continue
    
    slice_2d = sorted_points[start:end, :2]
    # 원점 거리 페널티 없이 순수하게 원 검출
    circle = fit_circle_ransac(slice_2d, n_iter=30, threshold=0.005, min_inliers=10)
    
    if circle:
        # 원점 거리 계산 및 final_score 계산
        dist_from_origin = np.linalg.norm(circle['center'] - origin)
        final_score = circle['score'] - dist_from_origin * 1.5
        
        circle_data.append({
            'z': z,
            'center': circle['center'],
            'radius': circle['radius'],
            'score': circle['score'],
            'final_score': final_score,
            'dist_from_origin': dist_from_origin
        })

if not circle_data:
    print("컵 원을 찾지 못했습니다.")
else:
    # final_score 기준으로 정렬하여 상위 5개 선택
    circle_data.sort(key=lambda x: x['final_score'], reverse=True)
    top_5_circles = circle_data[:5]
    center_axis = np.median([c['center'] for c in top_5_circles], axis=0)
    reference_z = top_5_circles[0]['z']
    
    print(f"중심축: ({center_axis[0]:.6f}, {center_axis[1]:.6f}), 기준 높이: {reference_z:.4f}m")
    
    # 2단계: 부피 계산을 위한 원 검출
    volume_data = []
    z_min, z_max = points[:, 2].min(), points[:, 2].max()
    
    # 기준 높이에서 시도
    ref_slice = get_slice(points, reference_z)
    ref_radius = detect_circle_at_axis(ref_slice[:, :2], center_axis) if len(ref_slice) >= MIN_POINTS_PER_SLICE else None
    
    if ref_radius:
        add_volume_data(volume_data, reference_z, ref_radius)
        search_heights(points, np.linspace(reference_z, z_max, 50)[1:], center_axis, volume_data)
        search_heights(points, np.linspace(reference_z, z_min, 50)[1:], center_axis, volume_data)
    else:
        z_range_full = np.concatenate([
            np.linspace(z_min, reference_z, 50),
            np.linspace(reference_z, z_max, 50)[1:]
        ])
        search_heights(points, z_range_full, center_axis, volume_data)
    
    # 3단계: 부피 계산
    if volume_data:
        volume_data.sort(key=lambda x: x['z'])
        total_volume = sum((volume_data[i]['area'] + volume_data[i+1]['area']) / 2 * 
                          (volume_data[i+1]['z'] - volume_data[i]['z']) 
                          for i in range(len(volume_data) - 1))
        
        print(f"\n단면 개수: {len(volume_data)}개")
        print(f"높이: {volume_data[0]['z']:.4f}m ~ {volume_data[-1]['z']:.4f}m")
        print(f"부피: {total_volume * 1000:.2f} mL ({total_volume * 1e6:.2f} cm³)")
    else:
        print("원을 찾지 못했습니다.")

중심축: (0.023763, 0.026822), 기준 높이: -0.0559m

단면 개수: 79개
높이: -0.1357m ~ 0.0858m
부피: 1.15 mL (1145.14 cm³)
